In [ ]:
%pip install osmapi pandas geopandas shapely matplotlib

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Leggere il file TSV
file_path = 'ita.tsv'
df = pd.read_csv(file_path, delimiter='\t',dtype=str)

# Stampa i nomi delle colonne per verificare
#print("Nomi delle colonne:", df.columns)

# Selezionare le colonne rilevanti e correggere eventuali spazi
df.columns = [col.strip() for col in df.columns]  # Rimuovere spazi extra dai nomi delle colonne
df = df[['geonameid', 'name', 'latitude', 'longitude', 'featureclass', 'featurecode', 'population', 'elevation']]


In [ ]:
# Aggiungere dettagli geografici basati sul 'featurecode'
def classify_geography(feature_code):
      feature_class = feature_class.strip()  # Rimuovere spazi extra
    feature_code = feature_code.strip()  # Rimuovere spazi extra
    
    if feature_class == 'T' and feature_code in ['MT', 'MTS']:  # Montagna, Montagne
        return 'montagna'
    elif feature_class == 'H' and feature_code in ['LK', 'LKS', 'LGN']:  # Lago, Laghi, Laguna
        return 'lago'
    elif feature_class == 'H' and feature_code in ['HBR', 'COVE']:  # Porto, Baia
        return 'mare'
    elif feature_class == 'L' and feature_code in ['PRK', 'RES']:  # Parco, Riserva
        return 'parco'
    elif feature_class == 'H' and feature_code in ['STM', 'WAD']:  # Fiume, Fiumara
        return 'fiume'
    elif feature_class == 'T' and feature_code in ['ISL', 'ISLS']:  # Isola, Isole
        return 'isola'
    elif feature_class == 'P' and feature_code in ['PPL', 'PPLA']:  # Città, Villaggi
        return 'città'
    else:
        return 'N.A.'

df['geography'] = df['featurecode'].apply(classify_geography)



In [ ]:

# Convertire lat/lon in punti geometrici
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# Filtrare per città con caratteristiche geografiche specifiche
gdf_filtered = gdf[gdf['geography'] != 'N.A.']


In [ ]:
gdf_filtered


In [ ]:

# Funzione per raccomandare luoghi basata su preferenze
def recommend_places(preferred_places, gdf_filtered):
    # Creare un DataFrame con i luoghi preferiti
    preferred_df = gdf_filtered[gdf_filtered['name'].isin(preferred_places)]
    
    # Calcolare la frequenza delle tipologie geografiche nei luoghi preferiti
    type_counts = preferred_df['geography'].value_counts()
    
    # Identificare la tipologia geografica più comune
    most_common_type = type_counts.idxmax()
    
    # Filtrare e raccomandare luoghi con la tipologia geografica più comune
    recommended_places = gdf_filtered[gdf_filtered['geography'] == most_common_type]
    
    return recommended_places




In [ ]:
# Lista di luoghi preferiti dell'utente
preferred_places = ['Firenze', 'Bari', 'Milano']

# Ottenere raccomandazioni
recommended_places = recommend_places(preferred_places, gdf_filtered)

# Visualizzare le prime righe dei luoghi raccomandati
print(recommended_places.head())

# Visualizzazione dei dati raccomandati
recommended_places.plot(marker='o', color='blue', markersize=5)
plt.title('Luoghi Raccomandati')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# Visualizzare le prime righe del DataFrame
print(gdf_filtered.head())

# Visualizzazione dei dati
gdf_filtered.plot(marker='o', color='red', markersize=5)
plt.title('Caratteristiche Geografiche delle Città Italiane')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# Salva i dati in un file CSV
output_file = 'italian_cities_with_geography.csv'
gdf_filtered.to_csv(output_file, index=False)

print(f"Dataset salvato in {output_file}")
